In [ ]:
import SimpleITK as sitk
import skimage
import skimage.measure
import numpy as np
import torch
import os

use_cuda = torch.cuda.is_available()
tensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
numpy = lambda x : x.detach().cpu().numpy()

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def load_nii_file(fname, threshold=.5):
    mask = sitk.GetArrayFromImage(sitk.ReadImage(fname))
    verts, faces, normals, values = skimage.measure.marching_cubes_lewiner(mask, threshold)
    return to_measure(verts, faces)
def to_measure(points, triangles):
    A, B, C = points[triangles[:, 0]], points[triangles[:, 1]], points[triangles[:, 2]]
    X = (A + B + C) / 3 
    S = np.sqrt(np.sum(np.cross(B - A, C - A) ** 2, 1)) / 2 
    return tensor(S / np.sum(S)), tensor(X)
if __name__ == "__main__":
    points = load_nii_file("BronchialTree.nii") 
    print(points[0].size())


In [ ]:
weights = points[0].numpy()
coordinates = points[1].numpy()


In [ ]:
max(coordinates[1])

In [ ]:
from plyfile import PlyElement,  PlyData
def write_ply(points, filename, text=True):
    points = [(points[i,0], points[i,1], points[i,2]) for i in range(points.shape[0])]
    vertex = np.array(points, dtype=[('x', 'f4'), ('y', 'f4'),('z', 'f4')])
    el = PlyElement.describe(vertex, 'vertex', comments=['vertices'])
    PlyData([el], text=text).write(filename)
write_ply(coordinates, "test.ply")

In [ ]:
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
def display_cloud(ax, measure, color):
    w_i, x_i = numpy(measure[0]), numpy(measure[1])
    ax.view_init(elev=110, azim=-90)
    weights = w_i / w_i.sum()
    ax.scatter(x_i[:, 0], x_i[:, 1], x_i[:, 2], s=25 * 500 * weights, c=color)
    ax.axes.set_xlim3d(left=-200, right=200)
    ax.axes.set_ylim3d(bottom=-200, top=200)
    ax.axes.set_zlim3d(bottom=-200, top=200)
ax = plt.subplot(111, projection='3d')
display_cloud(ax, points, "blue")

In [ ]:
import open3d as o3d
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(coordinates)
o3d.io.write_point_cloud("./data.ply", pcd)
o3d.visualization.draw_geometries([pcd])

In [ ]:
import os
import glob
import trimesh
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt
tf.random.set_seed(1234)


In [ ]:
DATA_DIR = tf.keras.utils.get_file(
    "modelnet.zip",
    "http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip",
    extract=True,
)
DATA_DIR = os.path.join(os.path.dirname(DATA_DIR), "ModelNet10")

In [ ]:
mesh = trimesh.load(os.path.join(DATA_DIR, "chair/train/chair_0001.off"))
mesh.show()

In [ ]:
points = mesh.sample(2048)

fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(111, projection="3d")
ax.scatter(points[:, 0], points[:, 1], points[:, 2])
ax.set_axis_off()
plt.show()

In [ ]:
print(len(points))

In [ ]:
def parse_dataset(num_points=2048):

    train_points = []
    train_labels = []
    test_points = []
    test_labels = []
    class_map = {}
    folders = glob.glob(os.path.join(DATA_DIR, "[!README]*"))

    for i, folder in enumerate(folders):
        print("processing class: {}".format(os.path.basename(folder)))
        class_map[i] = folder.split("/")[-1]
        train_files = glob.glob(os.path.join(folder, "train/*"))
        test_files = glob.glob(os.path.join(folder, "test/*"))

        for f in train_files:
            train_points.append(trimesh.load(f).sample(num_points))
            train_labels.append(i)

        for f in test_files:
            test_points.append(trimesh.load(f).sample(num_points))
            test_labels.append(i)

    return (
        np.array(train_points),
        np.array(test_points),
        np.array(train_labels),
        np.array(test_labels),
        class_map,
    )

In [ ]:
print(np.shape(test_points))

In [ ]:
NUM_POINTS = 2048
NUM_CLASSES = 10
BATCH_SIZE = 32

train_points, test_points, train_labels, test_labels, CLASS_MAP = parse_dataset(
    NUM_POINTS
)

In [ ]:
point_train = []
point_test = []
array = []
print(len(train_points))
for i in range(len(train_points)):
    array.append([0,0,0])
point_test = np.array(array)
point_test = np.expand_dims(point_test, axis=1)

In [ ]:
print(np.shape(point_test))

In [ ]:
def augment(points, label):
    points += tf.random.uniform(points.shape, -0.005, 0.005, dtype=tf.float64)
    points = tf.random.shuffle(points)
    return points, label



In [ ]:
def conv_bn(x, filters):
    x = layers.Conv1D(filters, kernel_size=1, padding="valid")(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)
def dense_bn(x, filters):
    x = layers.Dense(filters)(x)
    x = layers.BatchNormalization(momentum=0.0)(x)
    return layers.Activation("relu")(x)

In [ ]:
class OrthogonalRegularizer(keras.regularizers.Regularizer):
    def __init__(self, num_features, l2reg=0.001):
        self.num_features = num_features
        self.l2reg = l2reg
        self.eye = tf.eye(num_features)

    def __call__(self, x):
        x = tf.reshape(x, (-1, self.num_features, self.num_features))
        xxt = tf.tensordot(x, x, axes=(2, 2))
        xxt = tf.reshape(xxt, (-1, self.num_features, self.num_features))
        return tf.reduce_sum(self.l2reg * tf.square(xxt - self.eye))

In [ ]:
def tnet(inputs, num_features):

    bias = keras.initializers.Constant(np.eye(num_features).flatten())
    reg = OrthogonalRegularizer(num_features)

    x = conv_bn(inputs, 32)
    x = conv_bn(x, 64)
    x = conv_bn(x, 512)
    x = layers.GlobalMaxPooling1D()(x)
    x = dense_bn(x, 256)
    x = dense_bn(x, 128)
    x = layers.Dense(
        num_features * num_features,
        kernel_initializer="zeros",
        bias_initializer=bias,
        activity_regularizer=reg,
    )(x)
    feat_T = layers.Reshape((num_features, num_features))(x)
    return layers.Dot(axes=(2, 1))([inputs, feat_T])

In [ ]:
from keras.models import Sequential
from keras.layers import Concatenate
from keras.layers import Activation
from keras.layers import Dense
from keras.layers import Dropout

inputs = keras.Input(shape=(NUM_POINTS, 3))
x = tnet(inputs, 3)
x = conv_bn(x, 32)
x = conv_bn(x, 32)
x = tnet(x, 32)
x = conv_bn(x, 32)
x = conv_bn(x, 64)
x = conv_bn(x, 512)
x = layers.GlobalMaxPooling1D()(x)
x = dense_bn(x, 256)
x = layers.Dropout(0.3)(x)
x = dense_bn(x, 128)
x = layers.Dropout(0.3)(x)

a_input = keras.Input(shape=(1, 3))
a = layers.GlobalMaxPooling1D()(a_input)
a = dense_bn(a, 128)

print(np.shape(a))
print(np.shape(x))
mergedOutput = Concatenate()([a, x])
out = Dense(128, activation='relu')(mergedOutput)
out = Dropout(0.8)(out)
out = Dense(32, activation='sigmoid')(out)
out = Dense(NUM_CLASSES, activation='softmax')(out)



model = keras.Model(inputs=[inputs, a_input], outputs=out, name="pointnet")
model.summary()

In [ ]:


print(model.output_shape)




In [ ]:
print(train_dataset)

In [ ]:
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=["sparse_categorical_accuracy"],
)

model.fit(x = [train_points, point_test], y= [train_labels], epochs=20, validation_data=test_dataset)

In [ ]:
data = test_dataset.take(1)

points, labels = list(data)[0]
points = points[:8, ...]
labels = labels[:8, ...]

preds = model.predict(points)
preds = tf.math.argmax(preds, -1)

points = points.numpy()

fig = plt.figure(figsize=(15, 10))
for i in range(8):
    ax = fig.add_subplot(2, 4, i + 1, projection="3d")
    ax.scatter(points[i, :, 0], points[i, :, 1], points[i, :, 2])
    ax.set_title(
        "pred: {:}, label: {:}".format(
            CLASS_MAP[preds[i].numpy()], CLASS_MAP[labels.numpy()[i]]
        )
    )
    print("prediction: " + CLASS_MAP[preds[i].numpy()])
    print("label: " + CLASS_MAP[labels.numpy()[i]])
    ax.set_axis_off()
plt.show()


In [ ]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(coordinates)
o3d.io.write_point_cloud("./data.ply", pcd)



In [ ]:
import open3d as o3d


pcd = o3d.io.read_point_cloud("data.ply")
print(pcd)

In [ ]:
mesh2 = trimesh.load("data.ply")
mesh2.show()

In [ ]:
mesh2.show()

In [ ]:
downpcd = pcd.voxel_down_sample(voxel_size=7.409)
print(downpcd)

In [ ]:
o3d.visualization.draw_geometries([downpcd])

In [ ]:
down_points = np.asarray(downpcd.points)
down_points = np.expand_dims(down_points, axis=0)

In [ ]:
print(np.shape(down_points))

In [ ]:
print(len(test_points[0]))

In [ ]:
preds = model.predict(down_points)
preds = tf.math.argmax(preds, -1)


In [ ]:
print(type(preds[0].numpy()))

In [ ]:
print(CLASS_MAP[preds[0].numpy()])